# Introduction to Artificial Intelligence Through Pathfinding Algorithms Using Python


**Howell Dalton "Chip" McCullough IV**

Artificial Intelligence describes an application of Computer Science that seeks to mimic the human brain. Historically, intellegent systems are defined to act in usually one of four categories (Russel & Norving, 2010):

  * "thinking humanly"
  * "thinking rationally"
  * "acting humanly"
  * "acting rationally"

In introducing Artificial Intelligence, we will concern ourselves with pathfinding algorithms -- algorithms in which an intelligent agent seeks to find an ideal outcome from point A to point B. Particularly, we will be developing intellegent agents that will "act rationally", as the best path from point A from Point B may not be known, let alone exist. In this project, we will be using Python 3, and the default GUI library for Python, Tkinter. While Python can be written in a simple text editor, I would personally recommend using JetBrain's PyCharm Community Edition IDE. Install instructions for everything can be found in the Appendix.

## 1) Modeling a Maze

First things first, we need to construct a model for our intelligent agent to traverse. Because we want real solutions, we need to build a maze under the following parameters:

  1. The maze is perfect (i.e. solvable)
  2. The maze is randomly generated (so that we cannot hard code a solution)

### 1.a) Graphs, Vertices, and Edges

Finally we're about to begin writing code, but before we get into that, we need to go through a bit of a mathematical background, since we will be creating our maze by creating a randomly generated spanning tree. 

So, what's a tree? What does it mean for a tree to be a spanning tree? 

Followng the chain of jargon, a tree is simply a graph -- a mathematical collection of vertices, $V$, and edges, $E$, defined as $\mathcal{G} = (V, E)$. Specifically a tree is a graph in which all vertices have exactly one "parent" and may have up to $n$ children. A vertex, $v \in V$, defines a singular point in a graph that may or may not be connected to another vertex by an edge, $e \in E$.

Finally, a spanning tree is a tree constructed from a graph in which all edges are connected with a minimum number of edges, and the newly constructed graph is acyclic (does not contain any cycles).

With that, we will begin by creating a model for a vertex, the Node:

```python
import binascii  # Used to create a unique hexidecimal key for testing Node equality
import os        # Used to create a unique hexidecimal key for testing Node equality

class Node(object):
    """
    Node data structure, modeling a vertex in a graph, specifically a vertex in a grid graph.
    """
    def __init__(self):
        """
        Create the Node object with the following parameters
        """
        self.__north = None  # The node to the north of this node
        self.__south = None  # The node to the south of this node
        self.__east = None   # The node to the east of this node
        self.__west = None   # The node to the west of this node
        self.__degree = 0    # deg(v) -- Represents the number of edges connected to this node
        self.__visited = 0   # 1 if the node has been visited, o.w. 0
        self.__key = binascii.hexlify(os.urandom(32))  # Unique 32-bit hex key used for identifying nodes

    @property
    def north(self):
        """
        Get the node to the north of self
        :return: self.__north
        :rtype: Node || None
        """
        return self.__north

    @north.setter
    def north(self, north):
        """
        Set the node to the north of self to #north
        :param north: node to the north of self
        :type north: Node || None
        """
        self.__north = north

    @property
    def south(self):
        """
        Get the node to the south of self
        :return: self.__south
        :rtype: Node || None
        """
        return self.__south

    @south.setter
    def south(self, south):
        """
        Set the node to the south of self to #south
        :param north: node to the south of self
        :type north: Node || None
        """
        self.__south = south

    @property
    def east(self):
        """
        Get the node to the east of self
        :return: self.__east
        :rtype: Node || None
        """
        return self.__east

    @east.setter
    def east(self, east):
        """
        Set the node to the east of self to #east
        :param north: node to the east of self
        :type north: Node || None
        """
        self.__east = east

    @property
    def west(self):
        """
        Get the node to the west of self
        :return: self.__west
        :rtype: Node || None
        """
        return self.__west

    @west.setter
    def west(self, west):
        """
        Set the node to the west of self to #west
        :param north: node to the west of self
        :type north: Node || None
        """
        self.__west = west

    @property
    def degree(self) -> int:
        """
        Returns the degree of the node, deg(v)
        :return: self.__degree
        :rtype: int
        """
        return self.__degree

    @degree.setter
    def degree(self, deg: int):
        """
        Set the degree of this node to #deg
        :param deg: degree
        :type deg: int
        """
        self.__degree = deg

    @property
    def visited(self) -> int:
        """
        Returns 1 if the node has been visited, 0 o.w.
        :return: self.__visited
        :rtype: int
        """
        return self.__visited

    @visited.setter
    def visited(self, mark: int):
        """
        Sets the value of self.__visited to #mark
        :param mark: 1
        :type mark: int
        """
        self.__visited = mark

    @property
    def key(self) -> str:
        """
        Returns the unique key associated with self
        :return: self.__key
        :rtype: str
        """
        return self.__key

    def isNorthOf(self, node) -> bool:
        """
        Returns True if self is "north" of #node, False o.w.
        :param node: "base" node
        :type node: Node
        :rtype: bool
        """
        return self.__south is node

    def isSouthOf(self, node) -> bool:
        """
        Returns True if self is "south" of #node, False o.w.
        :param node: "base" node
        :type node: Node
        :rtype: bool
        """
        return self.__north is node

    def isEastOf(self, node) -> bool:
        """
        Returns True if self is "east" of #node, False o.w.
        :param node: "base" node
        :type node: Node
        :rtype: bool
        """
        return self.__west is node

    def isWestOf(self, node) -> bool:
        """
        Returns True if self is "west" of #node, False o.w.
        :param node: "base" node
        :type node: Node
        :rtype: bool
        """
        return self.__east is node

    def __eq__(self, other) -> bool:
        return self.key == other.key

    def __str__(self) -> str:
        return str(self.__visited)
```

Congrats! We've just implemented a basic model for a single vertex in our graph! Looking at the Node class above may still be a little confusing, so let's clarify a few things up:

  1. What are self.\_\_north, self.\_\_south, self.\_\_east, self.\_\_west?
    * self.\_\_XXXXX variables are private instance variables -- variables that all Node object inherently have. We could have just as easily defined them as self.north, self.south, etc., however, that would allow anything outside of the Node class we just created to access and modify the instance variables. By appending '\_\_' before the variable name, we've made the instance variables private, meaning only the Node object can make calls to self.\_\_XXXXX. We create north, south, east, and west instance variables because we will be creating a [grid graph](https://en.wikipedia.org/wiki/Lattice_graph). We could go into the mathematical definition of a lattice graph to describe a grid graph, but for the sake of this tutorial, we just need to understand that in a grid graph, all verteces are connected by at most four edges and have no diagonal connections, which creates an $(n \times m)$ structure that looks like a rectangular grid of width $n$ and height $m$.
  2. What is 'None'?
    * 'None' is Python's null constant. Described in the [Python documentation](https://docs.python.org/3/library/constants.html), None is the "sole value of the type `NoneType`. `None` is frequently used to represent the absence of a value" (2017). We set the above instance variables to 'None', simply because we don't know what will be north, south, east, or west of the newly created Node (or if there even will be anything).
  3. What are '@property' and '@XXXXX.setter'?
    * Using '@property' allows for access of private instance variables outside of the Node class, while '@XXXXX.setter' allows for setting the value of 'XXXXX' outside of the Node class.

If you've caught it by now, having the '.setter' for the north, south, east, and west variables seems a little insecure, since they can be changed after the graph has been created. Since this could potentially ruin the model of our grid graph, let's fix that. For every directional setter method, add the following:

```python
if self.__XXXXX is None:
```

This way, the new setter method for north now looks like:

```python
@north.setter
def north(self, north):
    if self.__north is None:
        self.north = north
```

Now, if north, south, east, or west have not respectively been set they will be set once, and remain that way until the script has been terminated.

### 1.b) Creating the Graph, $\mathcal{G}$

We now have a model of a vertex $v \in V$, but how exactly do we construct a graph using it? First, we need to decide on the dimensions of our grid. To keep things easy for now, let's let $n = m = 3$. Our next objective is to model our grid graph. Since a grid graph looks like an $(n \times m)$ rectangular grid, we'll model our graph by using a matrix, or a 2-dimensional array. We'll begin by making our function header:

```python
def generate_grid(height: int, width: int) -> list:
    """
    Generates and returns the model of a grid graph of height #height, and width #width.
    :param height: The height of our grid graph
    :type height: int
    :param width: The width of our grid graph
    :type width: int
    :return: The model of our graph as a 2-dimensional array
    :rtype: list
    """
```

Because we want this as general as possible, the only thing we need to supply are an arbitrary positive integer height and width, which for the time being will both be 3. Algorithmically, we need to do the following:

  1. Construct an empty graph, G
  2. Create $n$ rows length $m$ of Node objects 
  3. Link the Node object appropriately
  4. Append the rows to G
  5. Return G

Starting by creating G,

```python
    g = []
```

we need to create $n$ rows of length $m$,

```python
    for i in range(0, height):
        row = []
        for j in range(0, width):
```

consisting of Node objects.

```python
        n = Node()
        row.append(n)
```
Here is where it gets a little challenging. We now need to link the Nodes appropriately, but we need to avoid calling indexes in the matrix that don't exist. To remedy this, we only need to link nodes in a row, if there is more than one node in the row, and we only need to link nodes in the matrix if there are mulitple rows in the matrix, giving us:

```python
        if len(row) > 1:
            n.west = row[j-1]
            row[j-1].east = n
        g.append(row)
        if len(g) > 1:
            for cn in g[i]:
                cn.north = g[i-1][g[i].index(cn)]
                g[i-1][g[i].index(cn)].south = cn
```

Thus, our function that will generate the model of our grid will finally be:

```python
def generate_grid(height: int, width: int) -> list:
    """
    Generates and returns the model of a grid graph of height #height, and width #width.
    :param height: The height of our grid graph
    :type height: int
    :param width: The width of our grid graph
    :type width: int
    :return: The model of our graph as a 2-dimensional array
    :rtype: list
    """
    g = []
    for i in range(0, height):
        row = []
        for j in range(0, width):
            n = Node()
            row.append(n)
            if len(row) > 1:
                n.west = row[j-1]
                row[j-1].east = n
        g.append(row)
        if len(g) > 1:
            for cn in g[i]:
                cn.north = g[i-1][g[i].index(cn)]
                g[i-1][g[i].index(cn)].south = cn

    return g
```

This will return our data model of a graph in which all neighboring vertices are connected to each other. If we print our graph model to the console using $n = m = 3$, we should get the following output:

`[[0, 0, 0], [0, 0, 0], [0, 0, 0]]`

### 1.c) Maze Generation: Prim's Algorithm

Now that we have the grid created, it's time to actually make our maze. To do this, we will create a spanning tree from the model of our graph using Prim's Algorithm. Prim's Algorithm operates similarly to Dijkstra's Algorithm for finding the shortest paths between vertices in a graph. Prim's algorithm has the property that the edges in the set $\mathcal{G}'$ always form a single, spanning tree. (Cormen et.al., 2009). Prim's general algorithm for generating a minimum spanning tree is as follows:

```
MST-PRIM(G, w, r)
for each u in G.V
    u.key = INF
    u.pi = NIL
r.key = 0
Q = G.V
while Q != EMPTY
    u = EXTRACT-MIN(Q)
    for each v in G.Adj[u]
        if v in Q and w(u, v) < v.key
            v.pi = u
            v.key = w(u, v)
```

In plain English, what this is doing is looking at every vertex in $\mathcal{G}$, and creating a new edge between vertices $u, v \in V$ if the distance between $u$ and $v$ is minimum (determined by an arbitrary function $w(u, v)$. Now, this is the basic algorithm -- it doesn't necessarily fulfill what we need it to do, so we need to modify it a bit. Rather than creating a minimum spanning tree, we would rather create a random spanning tree, which fulfills both initial parameters, that:

  1. The maze is perfect, and
  2. The maze is randomly generated.

We'll begin to accomplish randomly generating the maze by picking a random vertex to start at. Starting by defining the function:

```python
import random

def mark(grid: list, order: list):
    subList = random.choice(grid)
    n = random.choice(subList)
    n.visited = 1
    order.append(n)
```

Now, by passing the grid and a list to the function __mark__, we can mark a random Node in our graph as a place to start generating our maze. Following our starting point, we need to branch off from the initial Node, so we'll focus on generating a list of potential next choices, called the fringe, or frontier.


```python
def generate_fringe(grid: list) -> list:
    mrk = []
    fng = []
    for row in grid:
        for n in row:
            if n.visited == 1:
                mrk.append(n)

    for n in mrk:
        if n.north is not None and n.north.visited == 0 and n.north not in fng:
            fng.append(n.north)
        if n.south is not None and n.south.visited == 0 and n.south not in fng:
            fng.append(n.south)
        if n.east is not None and n.east.visited == 0 and n.east not in fng:
            fng.append(n.east)
        if n.west is not None and n.west.visited == 0 and n.west not in fng:
            fng.append(n.west)

    return fng
```

Using the function described above, we can generate a set of unique Nodes that neighbor the marked Nodes already in the list. With this, we can expand the previous __mark__ function to handle an extra parameter:

```python
def mark(grid: list, order: list, fringe: list = None):
    if fringe is not None:
        n = random.choice(fringe)
        n.degree += 1
    else:
        subList = random.choice(grid)
        n = random.choice(subList)

    n.visited = 1
    order.append(n)
```

## 2) Depth First Search

```python
import tkinter

PX_HEIGHT = 10
PX_WIDTH = 10
BG_PASSED="#bb0000"


def depth_first_search(maze: list, root: tkinter.Tk = None):
    """
    Depth first search maze solver
    :param maze: (n x m) Maze matrix model
    :type maze: list
    :param root: Root GUI view
    :type root: tkinter.Tk
    """
    discovered = []
    stack = []
    start = (-1, -1)

    for i in range(0, len(maze)):
        if maze[i][0] != 0:
            start = (i, 0)
            break
    if start == (-1, -1):
        for j in range(0, len(maze[0])):
            if maze[0][j] != 0:
                start = (0, j)
                break

    print("Start found at: {0}".format(start))

    stack.append(start)

    while len(stack) > 0:
        loc = stack.pop()

        if root is not None:
            if len(discovered) > 0:
                tkinter.Canvas(root, bd=0, highlightthickness=0, bg=BG_PASSED,
                                height=PX_HEIGHT, width=PX_WIDTH).grid(row=discovered[-1][0], column=discovered[-1][1])
            tkinter.Canvas(root, bd=0, highlightthickness=0, bg='red',
                           height=PX_HEIGHT, width=PX_WIDTH).grid(row=loc[0], column=loc[1])
            root.update()

        if (len(maze) -1 in loc) or (len(maze[0]) - 1 in loc):
            break

        discovered.append(loc)

        north = (loc[0] - 1, loc[1])
        south = (loc[0] + 1, loc[1])
        east = (loc[0], loc[1] + 1)
        west = (loc[0], loc[1] - 1)

        if north[0] > 0 and north[0] < len(maze) and north[1] > 0 and north[1] < len(maze):
            if maze[north[0]][north[1]] != 0 and north not in discovered:
                stack.append(north)
        if south[0] > 0 and south[0] < len(maze) and south[1] > 0 and south[1] < len(maze):
            if maze[south[0]][south[1]] != 0 and south not in discovered:
                stack.append(south)
        if east[0] > 0 and east[0] < len(maze[0]) and east[1] > 0 and east[1] < len(maze[0]):
            if maze[east[0]][east[1]] != 0 and east not in discovered:
                stack.append(east)
        if west[0] > 0 and west[0] < len(maze[0]) and west[1] > 0 and west[1] < len(maze[0]):
            if maze[west[0]][west[1]] != 0 and west not in discovered:
                stack.append(west)
```

## 3) Breadth First Search

```python
import tkinter
from collections import deque

PX_HEIGHT = 10
PX_WIDTH = 10
BG_PASSED="#00bb00"


def breadth_first_search(maze: list, root: tkinter.Tk = None):
    """
    Breadth first search maze solver
    :param maze: (n x m) Maze matrix model
    :type maze: list
    :param root: Root GUI view
    :type root: tkinter.Tk
    """
    discovered = []
    queue = deque([])
    start = (-1, -1)

    for i in range(0, len(maze)):
        if maze[i][0] != 0:
            start = (i, 0)
            break
    if start == (-1, -1):
        for j in range(0, len(maze[0])):
            if maze[0][j] != 0:
                start = (0, j)
                break

    print("Start found at: {0}".format(start))

    queue.append(start)

    while len(queue) > 0:
        loc = queue.popleft()

        if root is not None:
            if len(discovered) > 0:
                tkinter.Canvas(root, bd=0, highlightthickness=0, bg=BG_PASSED,
                                height=PX_HEIGHT, width=PX_WIDTH).grid(row=discovered[-1][0], column=discovered[-1][1])
            tkinter.Canvas(root, bd=0, highlightthickness=0, bg='green',
                           height=PX_HEIGHT, width=PX_WIDTH).grid(row=loc[0], column=loc[1])
            root.update()

        if (len(maze) -1 in loc) or (len(maze[0]) - 1 in loc):
            break

        discovered.append(loc)

        north = (loc[0] - 1, loc[1])
        south = (loc[0] + 1, loc[1])
        east = (loc[0], loc[1] + 1)
        west = (loc[0], loc[1] - 1)

        if 0 < north[0] < len(maze) and 0 < north[1] < len(maze):
            if maze[north[0]][north[1]] != 0 and north not in discovered:
                queue.append(north)
        if 0 < south[0] < len(maze) and 0 < south[1] < len(maze):
            if maze[south[0]][south[1]] != 0 and south not in discovered:
                queue.append(south)
        if 0 < east[0] < len(maze[0]) and 0 < east[1] < len(maze[0]):
            if maze[east[0]][east[1]] != 0 and east not in discovered:
                queue.append(east)
        if 0 < west[0] < len(maze[0]) and 0 < west[1] < len(maze[0]):
            if maze[west[0]][west[1]] != 0 and west not in discovered:
                queue.append(west)
```

# References

 Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). The Algorithms of Kruskal and Prim. In Introduction to algorithms (pp. 631-642). Cambridge, MA: The MIT Press. 

Russell, S. J., & Norving, P. (2010). Introduction. In Artificial intelligence: A modern approach (pp. 1-33). New Jersey: Pearson. 